## Load Dataset

In [273]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [274]:
import pandas as pd
import numpy as np
from joblib import dump

In [275]:
df_raw =pd.read_csv('../data/raw/beer_reviews.csv')

In [276]:
df = df_raw.sample(n=5000, random_state=42)

In [277]:
df.shape

(5000, 13)

In [278]:
df_cleaned = df.copy()
columns_to_drop = ['brewery_id', 'beer_beerid', 'review_profilename']
df_cleaned.drop(columns_to_drop, axis=1, inplace=True)
df_cleaned.dropna(inplace=True)
df_cleaned.reset_index(drop=True, inplace=True)

In [279]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
num_cols = ['review_time', 'review_overall', 'review_aroma', 'review_appearance', 'review_palate', 'review_taste', 'beer_abv']
sc = StandardScaler()
df_cleaned[num_cols] = sc.fit_transform(df_cleaned[num_cols])

In [280]:
cat_cols = ['brewery_name', 'beer_style', 'beer_name']
# creating instance of labelencoder
le = LabelEncoder()
X_cat = pd.DataFrame()

for col in cat_cols:
    X_cat[col] = le.fit_transform(df_cleaned[col])

X_cat.columns = cat_cols

In [281]:
df_cleaned.drop(cat_cols, axis=1, inplace=True)

In [282]:
X = pd.concat([df_cleaned, X_cat], axis=1)
X.head()

review_time  review_overall  review_aroma  review_appearance  \
0     1.203123        0.227083      1.080284           1.031849   
1    -0.423751       -1.893147     -1.111049          -0.598979   
2    -0.040342        0.227083     -0.380605           0.216435   
3    -2.083433        0.933827      0.349839          -0.598979   
4    -0.730606        1.640570      0.349839           1.847264   

   review_palate  review_taste  beer_abv  brewery_name  beer_style  beer_name  
0      -0.403046     -0.436944  4.878019           598           4        332  
1      -1.143664     -1.820626 -0.609603           678          35       1891  
2      -0.403046     -0.436944 -0.864841           422          47         52  
3       0.337571      0.946737 -0.949921           173          36       1495  
4       1.818805      1.638578 -0.779762           918          78       2842

In [283]:
#set numpy=false
from src.data.sets import split_sets_random
X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(X, target_col='beer_style', test_ratio=0.2, to_numpy=False)

In [284]:
#Print the dimensions of `X_train`, `X_val`, `X_test`
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(2877, 9)
(959, 9)
(960, 9)


In [285]:
#from src.data.sets import load_sets
#X_train, y_train, X_val, y_val, X_test, y_test = load_sets(path='../data/processed/')

In [286]:
from src.models.pytorch import PytorchDataset

train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)

## Baseline Model

In [287]:
from src.models.null import NullModel
baseline_model = NullModel(target_type='classification')
y_base = baseline_model.fit_predict(y_train)

In [288]:
from src.models.performance import print_class_perf
print_class_perf(y_base, y_train, set_name='Training', average='weighted')

Accuracy Training: 0.08828640945429267
F1 Training: 0.014324335995777922


## Define Architecture

In [289]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [290]:
from src.models.pytorch import PytorchMultiClass
model = PytorchMultiClass(X_train.shape[1])

In [291]:
from src.models.pytorch import get_device
device = get_device()
model.to(device)

PytorchMultiClass(
  (layer_1): Linear(in_features=9, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=104, bias=True)
  (softmax): Softmax(dim=1)
)

In [292]:
#print architecture
print(model)

PytorchMultiClass(
  (layer_1): Linear(in_features=9, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=104, bias=True)
  (softmax): Softmax(dim=1)
)


## Train model

In [293]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [294]:
N_EPOCHS = 50
BATCH_SIZE = 30

In [295]:
from src.models.pytorch import test_classification, train_classification

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

Epoch: 0
	(train)	|	Loss: 0.1553	|	Acc: 0.7%
	(valid)	|	Loss: 0.1555	|	Acc: 0.0%
Epoch: 1
	(train)	|	Loss: 0.1555	|	Acc: 0.1%
	(valid)	|	Loss: 0.1555	|	Acc: 0.0%
Epoch: 2
	(train)	|	Loss: 0.1554	|	Acc: 0.4%
	(valid)	|	Loss: 0.1553	|	Acc: 0.6%
Epoch: 3
	(train)	|	Loss: 0.1554	|	Acc: 0.5%
	(valid)	|	Loss: 0.1553	|	Acc: 0.6%
Epoch: 4
	(train)	|	Loss: 0.1553	|	Acc: 0.6%
	(valid)	|	Loss: 0.1553	|	Acc: 0.6%
Epoch: 5
	(train)	|	Loss: 0.1552	|	Acc: 1.0%
	(valid)	|	Loss: 0.1553	|	Acc: 0.6%
Epoch: 6
	(train)	|	Loss: 0.1552	|	Acc: 1.1%
	(valid)	|	Loss: 0.1549	|	Acc: 1.8%
Epoch: 7
	(train)	|	Loss: 0.1550	|	Acc: 1.5%
	(valid)	|	Loss: 0.1549	|	Acc: 1.8%
Epoch: 8
	(train)	|	Loss: 0.1551	|	Acc: 1.1%
	(valid)	|	Loss: 0.1550	|	Acc: 1.5%
Epoch: 9
	(train)	|	Loss: 0.1551	|	Acc: 1.3%
	(valid)	|	Loss: 0.1550	|	Acc: 1.5%
Epoch: 10
	(train)	|	Loss: 0.1552	|	Acc: 1.0%
	(valid)	|	Loss: 0.1549	|	Acc: 1.8%
Epoch: 11
	(train)	|	Loss: 0.1550	|	Acc: 1.6%
	(valid)	|	Loss: 0.1549	|	Acc: 1.8%
Epoch: 12
	(train)	|	Loss:

In [252]:
torch.save(model, "../models/pytorch_1.pt")

## Test model

In [296]:
test_loss, test_acc = test_classification(test_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)
print(f'\tLoss: {test_loss:.4f}\t|\tAccuracy: {test_acc:.1f}')

	Loss: 0.1547	|	Accuracy: 0.0


In [297]:
import sktorch

ModuleNotFoundError: No module named 'sktorch'